In [ ]:
# Cell 1: Setup and Google Drive Access
import pandas as pd
import numpy as np
import glob
import requests
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

print("✅ Google Drive mounted successfully.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted successfully.


In [ ]:
# Add this new cell near the top of your notebook

from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import os

def calculate_all_metrics(y_true, y_pred):
    """Calculates MAE, MSE, RMSE, and R-squared."""
    mae = np.mean(np.abs(y_true - y_pred))
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    # R2 score requires multi-output='uniform_average' for multi-dimensional arrays
    r2 = r2_score(y_true, y_pred, multioutput='uniform_average')
    return mae, mse, rmse, r2

In [ ]:
# Cell 1: Installations
!pip install numpy torch networkx node2vec

In [ ]:
# Cell 2: Contents of lib/utils.py
import numpy as np
import torch
import random
import math

def init_seed(seed):
    torch.cuda.cudnn_enabled = False
    torch.backends.cudnn.deterministic = True
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

def metric(pred, label):
    with np.errstate(divide = 'ignore', invalid = 'ignore'):
        mask = np.not_equal(label, 0)
        mask = mask.astype(np.float32)
        mask /= np.mean(mask)
        mae = np.abs(np.subtract(pred, label)).astype(np.float32)
        rmse = np.square(mae)
        mape = np.divide(mae, label)
        mae = np.nan_to_num(mae * mask)
        mae = np.mean(mae)
        rmse = np.nan_to_num(rmse * mask)
        rmse = np.sqrt(np.mean(rmse))
        mape = np.nan_to_num(mape * mask)
        mape = np.mean(mape)
    return mae, rmse, mape

def seq2instance(data, P, F, H):
    num_step, dims = data.shape
    num_sample = num_step - H - P - F + 1
    x = np.zeros(shape = (num_sample, P, dims))
    y = np.zeros(shape = (num_sample, F, dims))
    z = np.zeros(shape = (num_sample, H, dims))
    for i in range(num_sample):
        z[i] = data[i : i + H]
        x[i] = data[i + H : i + H + P]
        y[i] = data[i + H + P : i + H + P + F]
    return z, x, y

def loadData(args):
    # --- Define the full paths to your data files on Google Drive ---
    DRIVE_DATA_PATH = "/content/drive/MyDrive/Youbike_Master_Project/YouBike_Demand_Forecast/data/"
    data_path = os.path.join(DRIVE_DATA_PATH, 'youbike.npz')
    SE_FILE = os.path.join(DRIVE_DATA_PATH, 'SE_youbike.txt')

    # --- The rest of the function uses these paths ---
    Traffic = np.load(data_path)['data']

    print("Initial loaded traffic Shape is: ", Traffic.shape)
    num_step = Traffic.shape[0]

    # train/val/test split
    train_steps = round(args.train_ratio * num_step)
    test_steps = round(args.test_ratio * num_step)
    val_steps = num_step - train_steps - test_steps
    train = Traffic[: train_steps]
    val = Traffic[train_steps : train_steps + val_steps]
    test = Traffic[-test_steps :]

    # Z, X, Y -> past, present, future
    # We only have one feature, so we squeeze the last dimension for seq2instance
    trainZ, trainX, trainY = seq2instance(train.squeeze(-1), args.P, args.F, args.H)
    valZ, valX, valY = seq2instance(val.squeeze(-1), args.P, args.F, args.H)
    testZ, testX, testY = seq2instance(test.squeeze(-1), args.P, args.F, args.H)

    # Normalization (using Z-score on occupancy_ratio)
    mean, std = np.mean(trainX), np.std(trainX)
    trainX = (trainX - mean) / std
    valX = (valX - mean) / std
    testX = (testX - mean) / std

    # Spatial embedding
    with open(SE_FILE, mode = 'r') as f:
        lines = f.readlines()
        temp = lines[0].split(' ')
        N, dims = int(temp[0]), int(temp[1])
        SE = np.zeros(shape = (N, dims), dtype = np.float32)
        for line in lines[1 :]:
            temp = line.split(' ')
            index = int(temp[0])
            SE[index] = temp[1 :]
    print("SE Shape is: ", SE.shape)

    # Temporal embedding (using 10-minute intervals = 144 steps per day)
    steps_per_day = 24 * 60 // 10
    dayofweek = np.reshape(np.arange(num_step) // steps_per_day % 7, newshape=(-1, 1))
    timeofday = np.reshape(np.arange(num_step) % steps_per_day, newshape=(-1,1))
    Time = np.concatenate((dayofweek, timeofday), axis = -1)

    # train/val/test for TE
    train_time = Time[: train_steps]
    val_time = Time[train_steps : train_steps + val_steps]
    test_time = Time[-test_steps :]

    trainTE = seq2instance(train_time, args.P, args.F, args.H)
    trainTE = np.concatenate(trainTE, axis = 1).astype(np.int32)
    valTE = seq2instance(val_time, args.P, args.F, args.H)
    valTE = np.concatenate(valTE, axis = 1).astype(np.int32)
    testTE = seq2instance(test_time, args.P, args.F, args.H)
    testTE = np.concatenate(testTE, axis = 1).astype(np.int32)

    return (trainZ, trainX, trainTE, trainY, valZ, valX, valTE, valY, testZ, testX, testTE, testY,
            SE, mean, std)

In [ ]:
# Cell 3 (Corrected): Contents of models/common_layer.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math # Make sure math is imported

class SpatialTransformerModel(torch.nn.Module):
    def __init__(self, K, d):
        super(SpatialTransformerModel, self).__init__()
        D = K*d
        self.FC_q = torch.nn.Linear(2*D, D)
        self.FC_k = torch.nn.Linear(2*D, D)
        self.FC_v = torch.nn.Linear(2*D, D)
        self.FC_o1 = torch.nn.Linear(D, D)
        self.FC_o2 = torch.nn.Linear(D, D)
        self.K = K
        self.d = d
        self.softmax = torch.nn.Softmax(dim=-1)

    def forward(self, X, STE, mask):
        X = torch.cat((X, STE), dim=-1)
        query = F.relu(self.FC_q(X))
        key = F.relu(self.FC_k(X))
        value = F.relu(self.FC_v(X))
        query = torch.cat(torch.split(query, self.d, dim=-1), dim=0)
        key = torch.cat(torch.split(key, self.d, dim=-1), dim=0)
        value = torch.cat(torch.split(value, self.d, dim=-1), dim=0)
        attention = torch.matmul(query, torch.transpose(key, 2, 3))
        attention /= (self.d ** 0.5)
        attention = self.softmax(attention)
        X = torch.matmul(attention, value)
        X = torch.cat(torch.split(X, X.shape[0]//self.K, dim=0), dim=-1)
        X = self.FC_o2(F.relu(self.FC_o1(X)))
        return X

class TemporalTransformerModel(torch.nn.Module):
    def __init__(self, K, d):
        super(TemporalTransformerModel, self).__init__()
        D = K*d
        self.FC_q = torch.nn.Linear(2*D, D)
        self.FC_k = torch.nn.Linear(2*D, D)
        self.FC_v = torch.nn.Linear(2*D, D)
        self.FC_o1 = torch.nn.Linear(D, D)
        self.FC_o2 = torch.nn.Linear(D, D)
        self.K = K
        self.d = d
        self.softmax = torch.nn.Softmax(dim=-1)

    def forward(self, X, STE, mask):
        X = torch.cat((X, STE), dim=-1)
        query = F.relu(self.FC_q(X))
        key = F.relu(self.FC_k(X))
        value = F.relu(self.FC_v(X))
        query = torch.cat(torch.split(query, self.d, dim=-1), dim=0)
        key = torch.cat(torch.split(key, self.d, dim=-1), dim=0)
        value = torch.cat(torch.split(value, self.d, dim=-1), dim=0)
        query = torch.transpose(query, 2, 1)
        key = torch.transpose(torch.transpose(key, 1, 2), 2, 3)
        value = torch.transpose(value, 2, 1)
        attention = torch.matmul(query, key)
        attention /= (self.d ** 0.5)
        if mask:
            batch_size = X.shape[0]
            num_step = X.shape[1]
            num_vertex = X.shape[2]
            mask_tensor = torch.ones(num_step, num_step).to(X.device)
            mask_tensor = torch.tril(mask_tensor)
            mask_tensor = torch.unsqueeze(torch.unsqueeze(mask_tensor, dim=0), dim=0)
            mask_tensor = mask_tensor.repeat(self.K * batch_size, num_vertex, 1, 1)
            mask_tensor = mask_tensor.to(torch.bool)
            attention = torch.where(mask_tensor, attention, -2 ** 15 + torch.tensor([1],dtype=torch.float32).to(X.device))
        attention = self.softmax(attention)
        X = torch.matmul(attention, value)
        X = torch.transpose(X, 2, 1)
        X = torch.cat(torch.split(X, X.shape[0]//self.K, dim=0), dim=-1)
        X = self.FC_o2(F.relu(self.FC_o1(X)))
        return X

class AttentionLayer(nn.Module):
    def __init__(self, flag, hidden_size, num_heads, layer_dropout=0.0, relu_dropout=0.0):
        super(AttentionLayer, self).__init__()
        if flag=='T': self.multi_head_attention = TemporalTransformerModel(num_heads, hidden_size//num_heads)
        if flag=='S': self.multi_head_attention = SpatialTransformerModel(num_heads, hidden_size//num_heads)
        self.positionwise_feed_forward = PositionwiseFeedForward(hidden_size, hidden_size, hidden_size, layer_config='ll', padding = 'both', dropout=relu_dropout)
        self.dropout = nn.Dropout(layer_dropout)
        self.layer_norm_mha = LayerNorm(hidden_size)
        self.layer_norm_ffn = LayerNorm(hidden_size)

    def forward(self, inputs, STE, mask):
        x = inputs
        x_norm = self.layer_norm_mha(x)
        y = self.multi_head_attention(x_norm, STE, mask)
        x = self.dropout(x + y)
        x_norm = self.layer_norm_ffn(x)
        y = self.positionwise_feed_forward(x_norm)
        y = self.dropout(x + y)
        return y

class PositionwiseFeedForward(nn.Module):
    def __init__(self, input_depth, filter_size, output_depth, layer_config='cc', padding='left', dropout=0.0):
        super(PositionwiseFeedForward, self).__init__()
        layers = []
        sizes = ([(input_depth, filter_size)] + [(filter_size, filter_size)]*(len(layer_config)-2) + [(filter_size, output_depth)])
        for lc, s in zip(list(layer_config), sizes):
            if lc == 'l': layers.append(nn.Linear(*s))
            else: raise ValueError("Unknown layer type {}".format(lc))
        self.layers = nn.ModuleList(layers)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs):
        x = inputs
        for i, layer in enumerate(self.layers):
            x = layer(x)
            if i < len(self.layers):
                x = self.relu(x)
                x = self.dropout(x)
        return x

class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(features))
        self.beta = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.gamma * (x - mean) / (std + self.eps) + self.beta

def _gen_embedding(length, channels, min_timescale=1.0, max_timescale=1.0e4):
    position = np.arange(length)
    num_timescales = channels // 2
    log_timescale_increment = ( math.log(float(max_timescale) / float(min_timescale)) / (float(num_timescales) - 1))

    # --- THIS LINE IS CORRECTED ---
    inv_timescales = min_timescale * np.exp(np.arange(num_timescales).astype(float) * -log_timescale_increment)

    scaled_time = np.expand_dims(position, 1) * np.expand_dims(inv_timescales, 0)
    signal = np.concatenate([np.sin(scaled_time), np.cos(scaled_time)], axis=1)
    signal = np.pad(signal, [[0, 0], [0, channels % 2]], 'constant', constant_values=[0.0, 0.0])
    signal =  signal.reshape([1, length, channels])
    return torch.from_numpy(signal).type(torch.FloatTensor)

In [ ]:
# Cell 4: Contents of models/spatial_adaptive_transformer.py
class SpatialAdaptiveTransformer(nn.Module):
    def __init__(self, epsilon, flag, hidden_size, max_step, num_heads, seq_length, input_dropout=0.0, layer_dropout=0.0,
                 relu_dropout=0.0, dhm=False):
        super(SpatialAdaptiveTransformer, self).__init__()
        self.position_embedding = _gen_embedding(seq_length, hidden_size)
        self.recurrent_embedding = _gen_embedding(max_step, hidden_size)
        self.max_step = max_step
        self.dhm = dhm
        self.att = AttentionLayer(flag, hidden_size, num_heads, layer_dropout, relu_dropout)
        self.layer_norm = LayerNorm(hidden_size)
        self.input_dropout = nn.Dropout(input_dropout)
        if(self.dhm): self.dhm_fn = DHM_basic_Spatial(epsilon, hidden_size)

    def forward(self, inputs, STE):
        x = self.input_dropout(inputs)
        if(self.dhm):
            x, (remainders,n_updates) = self.dhm_fn(x, inputs, STE, self.att, self.position_embedding, self.recurrent_embedding, self.max_step)
            return x, (remainders,n_updates)
        else:
            for l in range(self.max_step):
                x += self.position_embedding[:, :inputs.shape[1], :].type_as(inputs.data)
                x += self.recurrent_embedding[:, l, :].unsqueeze(1).repeat(1,inputs.shape[1],1).type_as(inputs.data)
                x = self.att(x, STE, None)
            return x, None

class DHM_basic_Spatial(nn.Module):
    def __init__(self, epsilon, hidden_size):
        super(DHM_basic_Spatial, self).__init__()
        self.sigma = nn.Sigmoid()
        self.p = nn.Linear(hidden_size,1)
        self.p.bias.data.fill_(1)
        self.threshold = 1 - epsilon

    def forward(self, state, inputs, STE, fn, pos_enc, recur_enc, max_step, encoder_output=None):
        B,T,N,D = state.shape
        device = state.device
        state = state.view(B*T,N,D)
        inputs = inputs.view(B*T,N,D)
        halting_probability = torch.zeros(B*T,N, device=device)
        remainders = torch.zeros(B*T,N, device=device)
        n_updates = torch.zeros(B*T,N, device=device)
        previous_state = torch.zeros_like(inputs, device=device)
        step = 0
        while( ((halting_probability<self.threshold) & (n_updates < max_step)).byte().any()):
            state = state + pos_enc[:, :inputs.shape[1], :].type_as(inputs.data).to(device)
            state = state + recur_enc[:, step, :].unsqueeze(1).repeat(1,inputs.shape[1],1).type_as(inputs.data).to(device)
            p = self.sigma(self.p(state)).squeeze(-1)
            still_running = (halting_probability < 1.0).float()
            new_halted = (halting_probability + p * still_running > self.threshold).float() * still_running
            still_running = (halting_probability + p * still_running <= self.threshold).float() * still_running
            halting_probability += p * still_running
            remainders += new_halted * (1 - halting_probability)
            halting_probability += new_halted * remainders
            n_updates += still_running + new_halted
            update_weights = p * still_running + new_halted * remainders
            state = state.view(B,T,N,D)
            state = fn(state, STE, None)
            state = state.view(B*T,N,D)
            previous_state = (state * update_weights.unsqueeze(-1)) + (previous_state * (1 - update_weights.unsqueeze(-1)))
            step+=1
        return previous_state.view(B,T,N,D), (remainders.view(B,T,N), n_updates.view(B,T,N))

In [ ]:
# Cell 5: Contents of models/temporal_adaptive_transformer.py
class TemporalAdaptiveTransformer(nn.Module):
    def __init__(self, epsilon, flag, hidden_size, max_step, num_heads, seq_length=100, input_dropout=0.0, layer_dropout=0.0,
                 relu_dropout=0.0, dhm=False):
        super(TemporalAdaptiveTransformer, self).__init__()
        self.position_embedding = _gen_embedding(seq_length, hidden_size)
        self.recurrent_embedding = _gen_embedding(max_step, hidden_size)
        self.max_step = max_step
        self.dhm = dhm
        self.att = AttentionLayer(flag, hidden_size, num_heads, layer_dropout, relu_dropout)
        self.layer_norm = LayerNorm(hidden_size)
        self.input_dropout = nn.Dropout(input_dropout)
        if(self.dhm): self.dhm_fn = DHM_basic_Temporal(epsilon, hidden_size)

    def forward(self, inputs, STE, mask):
        x = self.input_dropout(inputs)
        if(self.dhm):
            x, (remainders,n_updates) = self.dhm_fn(x, inputs, STE, self.att, self.position_embedding, self.recurrent_embedding, self.max_step, mask)
            return x, (remainders,n_updates)
        else:
            for l in range(self.max_step):
                x += self.position_embedding[:, :inputs.shape[1], :].type_as(inputs.data)
                x += self.recurrent_embedding[:, l, :].unsqueeze(1).repeat(1,inputs.shape[1],1).type_as(inputs.data)
                x = self.att(x, STE, mask)
            return x, None

class DHM_basic_Temporal(nn.Module):
    def __init__(self, epsilon, hidden_size):
        super(DHM_basic_Temporal, self).__init__()
        self.sigma = nn.Sigmoid()
        self.p = nn.Linear(hidden_size,1)
        self.p.bias.data.fill_(1)
        self.threshold = 1 - epsilon

    def forward(self, state, inputs, STE, fn, pos_enc, recur_enc, max_step, mask, encoder_output=None):
        B,T,N,D = state.shape
        device = state.device
        state = state.transpose(1,2).reshape(-1,T,D)
        inputs = inputs.transpose(1,2).reshape(-1,T,D)
        halting_probability = torch.zeros(inputs.shape[0],inputs.shape[1], device=device)
        remainders = torch.zeros(inputs.shape[0],inputs.shape[1], device=device)
        n_updates = torch.zeros(inputs.shape[0],inputs.shape[1], device=device)
        previous_state = torch.zeros_like(inputs, device=device)
        step = 0
        while( ((halting_probability<self.threshold) & (n_updates < max_step)).byte().any()):
            state = state + pos_enc[:, :inputs.shape[1], :].type_as(inputs.data).to(device)
            state = state + recur_enc[:, step, :].unsqueeze(1).repeat(1,inputs.shape[1],1).type_as(inputs.data).to(device)
            p = self.sigma(self.p(state)).squeeze(-1)
            still_running = (halting_probability < 1.0).float()
            new_halted = (halting_probability + p * still_running > self.threshold).float() * still_running
            still_running = (halting_probability + p * still_running <= self.threshold).float() * still_running
            halting_probability += p * still_running
            remainders += new_halted * (1 - halting_probability)
            halting_probability += new_halted * remainders
            n_updates += still_running + new_halted
            update_weights = p * still_running + new_halted * remainders
            state = state.view(B,N,T,D).transpose(1,2)
            state = fn(state, STE, mask)
            state = state.transpose(1,2).view(-1,T,D)
            previous_state = (state * update_weights.unsqueeze(-1)) + (previous_state * (1 - update_weights.unsqueeze(-1)))
            step+=1
        return previous_state.view(B,N,T,D).transpose(1,2), (remainders.view(B,N,T), n_updates.view(B,N,T))

In [ ]:
# Cell 6: Contents of models/model.py
class STEmbModel(torch.nn.Module):
    def __init__(self, SEDims, TEDims, OutDims, device):
        super(STEmbModel, self).__init__()
        self.TEDims = TEDims
        self.FC_se1 = torch.nn.Linear(SEDims, OutDims)
        self.FC_se2 = torch.nn.Linear(OutDims, OutDims)
        self.FC_te1 = torch.nn.Linear(TEDims, OutDims)
        self.FC_te2 = torch.nn.Linear(OutDims, OutDims)
        self.device = device

    def forward(self, SE, TE):
        SE = SE.unsqueeze(0).unsqueeze(0)
        SE = self.FC_se2(F.relu(self.FC_se1(SE)))
        dayofweek = F.one_hot(TE[..., 0], num_classes = 7)
        timeofday = F.one_hot(TE[..., 1], num_classes = self.TEDims-7)
        TE = torch.cat((dayofweek, timeofday), dim=-1)
        TE = TE.unsqueeze(2).type(torch.FloatTensor).to(self.device)
        TE = self.FC_te2(F.relu(self.FC_te1(TE)))
        sum_tensor = torch.add(SE, TE)
        return sum_tensor

class EntangleModel(torch.nn.Module):
    def __init__(self, K, d):
        super(EntangleModel, self).__init__()
        D = K*d
        self.FC_xs = torch.nn.Linear(D, D)
        self.FC_xt = torch.nn.Linear(D, D)
        self.FC_h1 = torch.nn.Linear(D, D)
        self.FC_h2 = torch.nn.Linear(D, D)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, HS, HT):
        XS = self.FC_xs(HS)
        XT = self.FC_xt(HT)
        z = self.sigmoid(torch.add(XS, XT))
        H = torch.add((z* HS), ((1-z)* HT))
        H = self.FC_h2(F.relu(self.FC_h1(H)))
        return H

class STATModel(torch.nn.Module):
    def __init__(self, K, d, epsilon, hidden_size, max_step, T, N, dhm):
        super(STATModel, self).__init__()
        self.spatialAdaptiveTransformer = SpatialAdaptiveTransformer(epsilon, 'S', hidden_size, max_step, K, N, dhm=dhm)
        self.temporalAdaptiveTransformer = TemporalAdaptiveTransformer(epsilon, 'T', hidden_size, max_step, K, T, dhm=dhm)
        self.entangle = EntangleModel(K, d)

    def forward(self, X, STE, mask):
        HS, dhm_S = self.spatialAdaptiveTransformer(X, STE)
        HT, dhm_T = self.temporalAdaptiveTransformer(X, STE, mask)
        H = self.entangle(HS, HT)
        return torch.add(X, H), dhm_S, dhm_T

class STTModel(torch.nn.Module):
    def __init__(self, K, d):
        super(STTModel, self).__init__()
        self.spatialTransformer = SpatialTransformerModel(K, d)
        self.temporalTransformer = TemporalTransformerModel(K, d)
        self.entangle = EntangleModel(K, d)

    def forward(self, X, STE, mask):
        HS = self.spatialTransformer(X, STE, mask)
        HT = self.temporalTransformer(X, STE, mask)
        H = self.entangle(HS, HT)
        return torch.add(X, H)

class CrossAttentionModel(torch.nn.Module):
    def __init__(self, K, d):
        super(CrossAttentionModel, self).__init__()
        D = K * d
        self.FC_Q_Cross = torch.nn.Linear(D, D)
        self.FC_K_Cross = torch.nn.Linear(D, D)
        self.FC_V_Cross = torch.nn.Linear(D, D)
        self.FC_Out1 = torch.nn.Linear(D, D)
        self.FC_Out2 = torch.nn.Linear(D, D)
        self.K = K
        self.d = d
        self.softmax = torch.nn.Softmax(dim=-1)

    def forward(self, X, STE_P, STE_Q):
        query = F.relu(self.FC_Q_Cross(STE_Q))
        key = F.relu(self.FC_K_Cross(STE_P))
        value = F.relu(self.FC_V_Cross(X))
        query = torch.cat(torch.split(query, self.d, dim=-1), dim=0)
        key = torch.cat(torch.split(key, self.d, dim=-1), dim=0)
        value = torch.cat(torch.split(value, self.d, dim=-1), dim=0)
        query = torch.transpose(query, 2, 1)
        key = torch.transpose(torch.transpose(key, 1, 2), 2, 3)
        value = torch.transpose(value, 2, 1)
        attention = torch.matmul(query, key)
        attention /= (self.d ** 0.5)
        attention = self.softmax(attention)
        X = torch.matmul(attention, value)
        X = torch.transpose(X, 2, 1)
        X = torch.cat(torch.split(X, X.shape[0]//self.K, dim=0), dim=-1)
        X = self.FC_Out2(F.relu(self.FC_Out1(X)))
        return X

class BISTAT(torch.nn.Module):
    def __init__(self, K, d, SEDims, TEDims, P, F, H, N, L, episilon, hidden_size, max_hop, dhm, device, input_dims=1):
        super(BISTAT, self).__init__()
        D = K*d
        self.P, self.F, self.H, self.L = P, F, H, L
        self.FC_1, self.FC_2 = torch.nn.Linear(input_dims, D), torch.nn.Linear(D, D)
        self.STEmb = STEmbModel(SEDims, TEDims, K*d, device)
        self.STATBlockEnc = torch.nn.ModuleList([STATModel(K, d, episilon, hidden_size, max_hop, P, N, dhm) for _ in range(self.L)])
        self.STATBlockDec1 = torch.nn.ModuleList([STATModel(K, d, episilon, hidden_size, max_hop, F, N, dhm) for _ in range(self.L)])
        self.STATBlockDec2 = torch.nn.ModuleList([STTModel(K, d) for _ in range(self.L)])
        self.CrossAttention1 = CrossAttentionModel(K, d)
        self.CrossAttention2 = CrossAttentionModel(K, d)
        self.FC_dec1_1, self.FC_dec1_2 = torch.nn.Linear(D, D), torch.nn.Linear(D, 1)
        self.FC_dec2_1, self.FC_dec2_2 = torch.nn.Linear(D, D), torch.nn.Linear(D, 1)

    def forward(self, X, SE, TE, flag):
        # Add the feature dimension back before the first layer
        X = X.unsqueeze(3)

        X = self.FC_2(F.relu(self.FC_1(X)))
        STE = self.STEmb(SE, TE)
        STE_H, STE_P, STE_F = STE[:, :self.H], STE[:, self.H:self.H+self.P], STE[:, self.H+self.P:]

        dhm0_S_enc, dhm1_S_enc, dhm0_T_enc, dhm1_T_enc = [], [], [], []

        X, dhm_S, dhm_T = self.STATBlockEnc[0](X, STE_P, mask=True)

        dhm0_S_enc.append(dhm_S[0]); dhm1_S_enc.append(dhm_S[1]); dhm0_T_enc.append(dhm_T[0]); dhm1_T_enc.append(dhm_T[1])
        X_enc_out_L1 = X

        for l in range(1, self.L):
            X, dhm_S, dhm_T = self.STATBlockEnc[l](X, STE_P, mask=True)
            dhm0_S_enc.append(dhm_S[0]); dhm1_S_enc.append(dhm_S[1]); dhm0_T_enc.append(dhm_T[0]); dhm1_T_enc.append(dhm_T[1])

        X_cross1_out = self.CrossAttention1(X, STE_P, STE_F)

        dhm0_S_dec, dhm1_S_dec, dhm0_T_dec, dhm1_T_dec = [], [], [], []

        X_dec1_out = X_cross1_out
        for net in self.STATBlockDec1:
            X_dec1_out, dhm_S, dhm_T = net(X_dec1_out, STE_F, mask=True)
            dhm0_S_dec.append(dhm_S[0]); dhm1_S_dec.append(dhm_S[1]); dhm0_T_dec.append(dhm_T[0]); dhm1_T_dec.append(dhm_T[1])

        dhm0_S = torch.sum(torch.stack(dhm0_S_enc + dhm0_S_dec, dim=0), dim=0)
        dhm1_S = torch.sum(torch.stack(dhm1_S_enc + dhm1_S_dec, dim=0), dim=0)
        dhm0_T = torch.sum(torch.stack(dhm0_T_enc + dhm0_T_dec, dim=0), dim=0)
        dhm1_T = torch.sum(torch.stack(dhm1_T_enc + dhm1_T_dec, dim=0), dim=0)

        X_dec1_out = self.FC_dec1_2(F.relu(self.FC_dec1_1(X_dec1_out)))

        if flag in ('train', 'val'):
            X_cross2_out = self.CrossAttention2(X_enc_out_L1, STE_P, STE_H)
            X_dec2_out = X_cross2_out
            for net in self.STATBlockDec2:
                 X_dec2_out = net(X_dec2_out, STE_H, mask=True)
            X_dec2_out = self.FC_dec2_2(F.relu(self.FC_dec2_1(X_dec2_out)))
            return X_dec1_out.squeeze(3), X_dec2_out.squeeze(3), dhm0_S, dhm1_S, dhm0_T, dhm1_T

        # --- THIS IS THE CORRECTED LINE ---
        # The original code returns 5 items for the test flag, not 6.
        if flag == 'test':
            return X_dec1_out.squeeze(3), dhm0_S, dhm1_S, dhm0_T, dhm1_T

def mae_loss(pred, label, device):
    mask = (label != 0).type(torch.FloatTensor).to(device)
    mask /= torch.mean(mask)
    mask[torch.isnan(mask)] = 0
    loss = torch.abs(pred - label) * mask
    loss[torch.isnan(loss)] = 0
    return torch.mean(loss)

In [ ]:
# Cell 7: Preprocessing 1 - Create Adjacency Matrix
import networkx as nx
import csv
import os

def get_adjacency_matrix(distance_df_filename, num_of_vertices, normalized_k=0.1):
    A = np.zeros((int(num_of_vertices), int(num_of_vertices)), dtype=np.float32)
    distanceA = np.full((int(num_of_vertices), int(num_of_vertices)), np.inf, dtype=np.float32)

    with open(distance_df_filename, 'r') as f:
        f.readline() # Skip header
        reader = csv.reader(f)
        for row in reader:
            if len(row) != 3: continue
            i, j, distance = int(row[0]), int(row[1]), float(row[2])
            distanceA[i, j] = distance
            distanceA[j, i] = distance # Make it symmetric

    for i in range(num_of_vertices):
        distanceA[i, i] = 0

    std = np.std(distanceA[distanceA != np.inf])
    adj = np.exp(-np.square(distanceA / std))
    adj[adj < normalized_k] = 0
    return adj

# --- CONFIGURE AND RUN ---
NUM_STATIONS = 105 # This should match your dataset

# <-- UPDATE THESE LINES -->
DRIVE_DATA_PATH = "/content/drive/MyDrive/Youbike_Master_Project/YouBike_Demand_Forecast/data/"
DISTANCE_FILE = os.path.join(DRIVE_DATA_PATH, 'youbike_distances.csv')
ADJ_FILE = os.path.join(DRIVE_DATA_PATH, 'youbike_adj.npy')

# This part remains the same
adjacency_matrix = get_adjacency_matrix(DISTANCE_FILE, NUM_STATIONS)
np.save(ADJ_FILE, adjacency_matrix)
print(f"Adjacency matrix saved to {ADJ_FILE}")

Adjacency matrix saved to /content/drive/MyDrive/Youbike_Master_Project/YouBike_Demand_Forecast/data/youbike_adj.npy


In [ ]:
# Cell 8: Preprocessing 2 - Generate Spatial Embeddings (SE)
from node2vec import Node2Vec

DRIVE_DATA_PATH = "/content/drive/MyDrive/Youbike_Master_Project/YouBike_Demand_Forecast/data/"
ADJ_FILE = os.path.join(DRIVE_DATA_PATH, 'youbike_adj.npy')
SE_FILE = os.path.join(DRIVE_DATA_PATH, 'SE_youbike.txt')

adj_mx = np.load(ADJ_FILE)
graph = nx.from_numpy_array(adj_mx)

# You can tune these node2vec parameters for better performance
node2vec = Node2Vec(graph, dimensions=64, walk_length=30, num_walks=200, workers=4)
model = node2vec.fit(window=10, min_count=1, batch_words=4)

model.wv.save_word2vec_format(SE_FILE)
print(f"Spatial embeddings saved to {SE_FILE}")

Computing transition probabilities:   0%|          | 0/105 [00:00<?, ?it/s]

In [ ]:
# Cell 9: Main Script - Configuration
import time
import datetime

class Args:
    # Data and dimensions
    time_slot = 10
    H = 6          # History steps (e.g., 6 10-min intervals = 1 hour back)
    P = 6          # Present steps (input sequence length, 1 hour)
    F = 6          # Future steps to predict (1 hour ahead)
    L = 2           # Number of STAT layers
    K = 3           # Number of attention heads
    d = 8           # Dimensions of each head's output
    dataset = 'youbike' # This should match your data file names
    train_ratio = 0.7
    val_ratio = 0.1
    test_ratio = 0.2

    # Training hyperparameters
    batch_size = 128
    max_epoch = 10
    patience = 10
    learning_rate = 0.001
    weight_decay = 0.0001

    # Model-specific parameters
    dhm = True      # Whether to use the Dynamic Halting Module
    max_step = 6    # Max recurrent steps for DHM
    epsilon = 0.0001
    recollection_weight = 0.01 # Loss weight for the history decoder
    dhm_weight = 0.001         # Penalty weight for the DHM

    # System
    seed = 10

args = Args()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set seed for reproducibility
init_seed(args.seed)

In [ ]:
# Cell 10: Main Script - Load Data
print('Loading data...')
# This calls the loadData function defined in Cell 2
(trainZ, trainX, trainTE, trainY, valZ, valX, valTE, valY, testZ, testX, testTE, testY, SE,
 mean, std) = loadData(args)

print(f'trainX shape: {trainX.shape}, trainY shape: {trainY.shape}')
print(f'valX shape: {valX.shape}, valY shape: {valY.shape}')
print(f'testX shape: {testX.shape}, testY shape: {testY.shape}')
print('Data loaded!')

# Transform data to PyTorch tensors
trainZ = torch.FloatTensor(trainZ).to(device)
trainX = torch.FloatTensor(trainX).to(device)
trainTE = torch.LongTensor(trainTE).to(device)
trainY = torch.FloatTensor(trainY).to(device)
valZ = torch.FloatTensor(valZ).to(device)
valX = torch.FloatTensor(valX).to(device)
valTE = torch.LongTensor(valTE).to(device)
valY = torch.FloatTensor(valY).to(device)
testZ = torch.FloatTensor(testZ).to(device)
testX = torch.FloatTensor(testX).to(device)
testTE = torch.LongTensor(testTE).to(device)
testY = torch.FloatTensor(testY).to(device)
SE = torch.FloatTensor(SE).to(device)

In [ ]:
# Cell 11 (Fully Corrected): Main Script - Training and Validation Loop

# --- Initialize Model and Optimizer ---
TEmbsize = (24 * 60 // 10) + 7 # Using 10-minute interval
hidden_size = args.K * args.d
num_sensor = trainX.shape[-1]

bi_stat = BISTAT(
    args.K, args.d, SE.shape[1], TEmbsize, args.P, args.F, args.H,
    num_sensor, args.L, args.epsilon, hidden_size, args.max_step, args.dhm, device,
    input_dims=1 # FIX 1: Explicitly set input features to 1
).to(device)

optimizer = torch.optim.Adam(bi_stat.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)

# Initialize model weights
for p in bi_stat.parameters():
   if p.dim() > 1: nn.init.xavier_uniform_(p)
   else: nn.init.uniform_(p)

# --- Training ---
print('**** Training Model ****')
num_train = trainX.shape[0]
num_val = valX.shape[0]
wait = 0
val_loss_min = np.inf

# --- Lists to store metrics for each epoch ---
val_mae_per_epoch = []
val_mse_per_epoch = []
val_rmse_per_epoch = []
val_r2_per_epoch = []

# --- Define Checkpoint Path ---
CHECKPOINT_PATH = "/content/drive/MyDrive/Youbike_Master_Project/BiSTAT_Checkpoints/"
os.makedirs(CHECKPOINT_PATH, exist_ok=True) # Creates the directory if needed
MODEL_FILE = os.path.join(CHECKPOINT_PATH, f'BISTAT_{args.dataset}.pt')

for epoch in range(args.max_epoch):
    if wait >= args.patience:
        print(f'Early stopping at epoch: {epoch}')
        break

    # --- Training Phase ---
    train_loss = 0
    num_batch = math.ceil(num_train / args.batch_size)
    bi_stat.train()
    for batch_idx in range(num_batch):
        optimizer.zero_grad()
        start_idx = batch_idx * args.batch_size
        end_idx = min(num_train, (batch_idx + 1) * args.batch_size)
        batchX, batchTE = trainX[start_idx:end_idx], trainTE[start_idx:end_idx]
        batchlabel1, batchlabel2 = trainY[start_idx:end_idx], trainZ[start_idx:end_idx]
        pred1, pred2, dhm0_S, dhm1_S, dhm0_T, dhm1_T = bi_stat(batchX, SE, batchTE, flag='train')
        loss_dec1 = mae_loss(pred1, batchlabel1, device)
        loss_dec2 = mae_loss(pred2, batchlabel2, device)
        loss_dhm = torch.mean(dhm0_S + dhm1_S) + torch.mean(dhm0_T + dhm1_T)
        batchloss = loss_dec1 + args.recollection_weight * loss_dec2 + args.dhm_weight * loss_dhm
        batchloss.backward()
        optimizer.step()
        train_loss += batchloss.item() * (end_idx - start_idx)
    train_loss /= num_train

    # --- Validation Phase ---
    bi_stat.eval()
    val_preds, val_trues = [], []
    with torch.no_grad():
        for batch_idx in range(math.ceil(num_val / args.batch_size)):
            start_idx = batch_idx * args.batch_size
            end_idx = min(num_val, (batch_idx + 1) * args.batch_size)
            batchX, batchTE, batchlabel1 = valX[start_idx:end_idx], valTE[start_idx:end_idx], valY[start_idx:end_idx]
            pred1, _, _, _, _, _ = bi_stat(batchX, SE, batchTE, flag='val')
            val_preds.append(pred1.cpu().numpy())
            val_trues.append(batchlabel1.cpu().numpy())

    val_preds = np.concatenate(val_preds, axis=0)
    val_trues = np.concatenate(val_trues, axis=0)

    # FIX 2: Reshape 3D arrays to 2D for scikit-learn metrics
    num_samples = val_trues.shape[0]
    val_trues_2d = val_trues.reshape(num_samples, -1)
    val_preds_2d = val_preds.reshape(num_samples, -1)

    # Calculate all metrics on the reshaped 2D arrays
    mae, mse, rmse, r2 = calculate_all_metrics(val_trues_2d, val_preds_2d)

    val_mae_per_epoch.append(mae)
    val_mse_per_epoch.append(mse)
    val_rmse_per_epoch.append(rmse)
    val_r2_per_epoch.append(r2)

    print(f"--- Epoch {epoch+1}/{args.max_epoch} ---")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val Metrics -> MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}")

    if mae <= val_loss_min:
        print(f'---> Validation MAE decreased from {val_loss_min:.4f} to {mae:.4f}, saving model.')
        wait = 0
        val_loss_min = mae
        torch.save(bi_stat.state_dict(), MODEL_FILE)
    else:
        wait += 1

**** Training Model ****


ValueError: Found array with dim 3. None expected <= 2.

In [ ]:
# The Final, Complete Cell for Testing, Analysis, and Plotting

# ===================================================================
# PART 0: Load Timestamps for Plotting
# ===================================================================
print("**** Loading Timestamps for Plotting ****")
# This is the path to your ORIGINAL feature-engineered data file
TIMESTAMP_FILE_PATH = "/content/drive/MyDrive/Youbike_Master_Project/YouBike_Demand_Forecast/data/gongguan_model_ready_features.parquet.gz"

# Efficiently load only the 'time' column into a new dataframe
timestamps_df = pd.read_parquet(TIMESTAMP_FILE_PATH, columns=['time'])


# ===================================================================
# PART 1: Run Predictions on the Test Set
# ===================================================================
print('\n**** Testing Model ****')
bi_stat.load_state_dict(torch.load(MODEL_FILE))
print(f'Model restored from {MODEL_FILE}!')

num_test = testX.shape[0]
testPred = []

bi_stat.eval()
with torch.no_grad():
    for batch_idx in range(math.ceil(num_test / args.batch_size)):
        start_idx = batch_idx * args.batch_size
        end_idx = min(num_test, (batch_idx + 1) * args.batch_size)
        batchX, batchTE = testX[start_idx:end_idx], testTE[start_idx:end_idx]
        batchpred, _, _, _, _ = bi_stat(batchX, SE, batchTE, flag='test')
        testPred.append(batchpred.detach().cpu().numpy())

testPred = np.concatenate(testPred, axis=0)
testY_truth = testY.cpu().numpy()


# ===================================================================
# PART 2: Calculate and Print All Metrics
# ===================================================================
# --- Overall Average Metrics ---
num_test_samples = testY_truth.shape[0]
testY_truth_2d = testY_truth.reshape(num_test_samples, -1)
testPred_2d = testPred.reshape(num_test_samples, -1)
mae, mse, rmse, r2 = calculate_all_metrics(testY_truth_2d, testPred_2d)
print('\n--- Overall Test Performance (Average of all horizons) ---')
print(f'Overall --> MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}')

# --- Per-Horizon Metrics ---
print('\n--- Performance per Prediction Horizon ---')
for q in range(args.F):
    y_true_horizon = testY_truth[:, q, :]
    y_pred_horizon = testPred[:, q, :]
    mae_h, mse_h, rmse_h, r2_h = calculate_all_metrics(y_true_horizon, y_pred_horizon)
    print(f'Horizon {q+1:02d} ----> MAE: {mae_h:.4f}, MSE: {mse_h:.4f}, RMSE: {rmse_h:.4f}, R²: {r2_h:.4f}')


# ===================================================================
# PART 3: Find Best and Worst Performing Stations
# ===================================================================
print("\n**** Analyzing Performance per Station ****")
station_errors = np.mean(np.abs(testPred - testY_truth), axis=(0, 1))
sorted_station_indices = np.argsort(station_errors)
best_3_indices = sorted_station_indices[:3]
worst_3_indices = sorted_station_indices[-3:]

print(f"\nTop 3 Best Performing Stations (by MAE):")
for i, idx in enumerate(best_3_indices):
    print(f"  {i+1}. Station Index: {idx}, MAE: {station_errors[idx]:.4f}")

print(f"\nTop 3 Worst Performing Stations (by MAE):")
for i, idx in enumerate(worst_3_indices[::-1]): # Reverse to show worst first
    print(f"  {i+1}. Station Index: {idx}, MAE: {station_errors[idx]:.4f}")


# ===================================================================
# PART 4: Create the Continuous, Sliding-Window Plot
# ===================================================================
print("\n**** Generating Continuous Forecast Plot for a Single Horizon ****")

# --- Configuration ---
station_to_plot_idx = best_3_indices[0] # Plot the best station
horizon_to_plot = 5 # This is the t+6 forecast (1 hour ahead)

# --- Extract the specific horizon data ---
predictions_continuous = testPred[:, horizon_to_plot, station_to_plot_idx]
actuals_continuous = testY_truth[:, horizon_to_plot, station_to_plot_idx]

# --- Reconstruct Timestamps using the `timestamps_df` we just loaded ---
num_step = timestamps_df['time'].nunique()
train_steps = round(args.train_ratio * num_step)
val_steps = round(args.val_ratio * num_step)
test_start_index = train_steps + val_steps
first_target_timestamp_index = test_start_index + args.H + args.P + horizon_to_plot
all_timestamps = np.sort(pd.to_datetime(timestamps_df['time']).unique())
end_timestamp_index = first_target_timestamp_index + len(actuals_continuous)
plot_timestamps = all_timestamps[first_target_timestamp_index:end_timestamp_index]

# --- Create the Plot ---
plt.figure(figsize=(20, 6))
plt.plot(plot_timestamps, actuals_continuous, label='Actual Values', color='blue')
plt.plot(plot_timestamps, predictions_continuous, label='Predicted Values', color='green', linestyle='--')
plt.title(f'Continuous t+{horizon_to_plot+1} Forecast for Station Index {station_to_plot_idx}', fontsize=16)
plt.xlabel('Date and Time')
plt.ylabel('Occupancy Ratio')
plt.ylim(0, 1)
plt.legend()
plt.grid(True)
plt.show()


**** Testing Model ****
Model restored from /content/drive/MyDrive/Youbike_Master_Project/BiSTAT_Checkpoints/BISTAT_youbike.pt!

--- Overall Test Performance (Average of all horizons) ---
Overall --> MAE: 0.0895, MSE: 0.0257, RMSE: 0.1603, R²: 0.6873

--- Performance per Prediction Horizon ---
Horizon 01 ----> MAE: 0.0524, MSE: 0.0105, RMSE: 0.1023, R²: 0.8734
Horizon 02 ----> MAE: 0.0714, MSE: 0.0175, RMSE: 0.1321, R²: 0.7882
Horizon 03 ----> MAE: 0.0860, MSE: 0.0236, RMSE: 0.1536, R²: 0.7134
Horizon 04 ----> MAE: 0.0985, MSE: 0.0291, RMSE: 0.1706, R²: 0.6458
Horizon 05 ----> MAE: 0.1095, MSE: 0.0343, RMSE: 0.1853, R²: 0.5817
Horizon 06 ----> MAE: 0.1193, MSE: 0.0392, RMSE: 0.1981, R²: 0.5210


In [ ]:
# ADD this new cell after the testing cell

print("\n**** Analyzing Performance per Station ****")

# Calculate MAE for each station individually across all samples and horizons
station_errors = np.mean(np.abs(testPred - testY_truth), axis=(0, 1))

# Get the indices of the stations, sorted by their error
sorted_station_indices = np.argsort(station_errors)

best_3_indices = sorted_station_indices[:3]
worst_3_indices = sorted_station_indices[-3:]

print(f"\nTop 3 Best Performing Stations (by MAE):")
for i, idx in enumerate(best_3_indices):
    print(f"  {i+1}. Station Index: {idx}, MAE: {station_errors[idx]:.4f}")

print(f"\nTop 3 Worst Performing Stations (by MAE):")
for i, idx in enumerate(worst_3_indices[::-1]): # Reverse to show worst first
    print(f"  {i+1}. Station Index: {idx}, MAE: {station_errors[idx]:.4f}")

# --- Plotting ---
# Let's pick a sample prediction from the middle of the test set to visualize
sample_idx_to_plot = len(testPred) // 2
horizons = np.arange(1, args.F + 1)

# Plot best stations
fig, axes = plt.subplots(1, 3, figsize=(20, 5), sharey=True)
fig.suptitle('3 Best Performing Stations (Sample Forecast)', fontsize=16)
for i, station_idx in enumerate(best_3_indices):
    axes[i].plot(horizons, testY_truth[sample_idx_to_plot, :, station_idx], 'o-', label='Actual')
    axes[i].plot(horizons, testPred[sample_idx_to_plot, :, station_idx], 'x-', label='Predicted')
    axes[i].set_title(f"Station Index {station_idx}\nMAE: {station_errors[station_idx]:.4f}")
    axes[i].set_xlabel("Forecast Horizon (10-min steps)")
    axes[i].set_ylabel("Occupancy Ratio")
    axes[i].legend()
    axes[i].grid(True)
plt.show()

# Plot worst stations
fig, axes = plt.subplots(1, 3, figsize=(20, 5), sharey=True)
fig.suptitle('3 Worst Performing Stations (Sample Forecast)', fontsize=16)
for i, station_idx in enumerate(worst_3_indices):
    axes[i].plot(horizons, testY_truth[sample_idx_to_plot, :, station_idx], 'o-', label='Actual')
    axes[i].plot(horizons, testPred[sample_idx_to_plot, :, station_idx], 'x-', label='Predicted')
    axes[i].set_title(f"Station Index {station_idx}\nMAE: {station_errors[station_idx]:.4f}")
    axes[i].set_xlabel("Forecast Horizon (10-min steps)")
    axes[i].set_ylabel("Occupancy Ratio")
    axes[i].legend()
    axes[i].grid(True)
plt.show()

In [ ]:
# ADD this new cell to save the results

print("\n**** Saving Test Set Predictions and Actuals ****")

# Define the path on your Google Drive
output_path = "/content/drive/MyDrive/Youbike_Master_Project/predictions/BiSTAT/"
os.makedirs(output_path, exist_ok=True) # Ensure directory exists

# Save the arrays to a single .npz file
np.savez(
    os.path.join(output_path, 'bistat_predictions.npz'),
    predictions=testPred,
    actuals=testY_truth
)

print(f"✅ Predictions and actuals saved to: {os.path.join(output_path, 'bistat_predictions.npz')}")